<a href="https://colab.research.google.com/github/AeroEng16/TennisPrediction/blob/main/playerDataScraping_ATP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import re
import json
import datetime
import pandas as pd


In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}

apiLink = "https://atptour.com/en/-/www/rank/sglroll/200?v=1"

individualPlayerBaseURL = "https://www.atptour.com/en/-/www/stats/PLAYERID/YEAR/COURTTYPE?v=1"
individualPlayerBaseURL_Physicals = "https://www.atptour.com/en/-/www/players/hero/PLAYERID?v=1"



year="all"  #year (e.g. 2024) or all
courtType = "all" #grass,clay,hard, carpet or all
allCourts = ["Grass","Clay","Hard","Carpet","all"]
playerDataSrc = requests.get(apiLink,headers=headers).json()



In [ ]:
playerData = []
for i in playerDataSrc:


    filteredDataSet = {key: value for key, value in zip(i.keys(), i.values()) if "RankNo" in key or "Name" in key or "Points" in key or "PlayerId" in key or "Movement" in key}
    # Extract Player Physical/Ranking Stats
    localURL_Physicals = individualPlayerBaseURL_Physicals.replace("PLAYERID",filteredDataSet["PlayerId"])
    responseIndividualPlayer_Physicals = requests.get(localURL_Physicals,headers=headers).json()
    relevantPlayerStats = {key: value for key, value in zip(responseIndividualPlayer_Physicals.keys(), responseIndividualPlayer_Physicals.values()) if "Age" in key or "HeightCm" in key or "WeightKg" in key or "PlayHand" in key or "SglHiRank" in key or "SglCareerWon" in key or "SglCareerLost" in key or "SglYtdWon" in key or "SglYtdLost" in key or "SglHiRankDate" in key}
    relevantPlayerStats["PlayHand"] = relevantPlayerStats["PlayHand"]['Id']


    print(filteredDataSet["PlayerId"])

    for k,j in enumerate(["PLAYERID","YEAR","COURTTYPE"]):
        if k==0:
            localURL = individualPlayerBaseURL.replace(j,filteredDataSet["PlayerId"])
        elif k == 1:
            localURL = localURL.replace(j,"all")
        elif k == 2:
            localURL = localURL.replace(j,courtType)

    responseIndividualPlayer = requests.get(localURL,headers=headers)

    firstStatYear = responseIndividualPlayer.json()["FirstStatYear"]
    lastStatYear  = responseIndividualPlayer.json()["LastStatYear"]

    yearRange = [str(i) for i in range(firstStatYear,lastStatYear+1)]
    yearRange.append('all')

    for l in yearRange:
      for m in allCourts:
        for k,j in enumerate(["PLAYERID","YEAR","COURTTYPE"]):
          if k==0:
              localURL = individualPlayerBaseURL.replace(j,filteredDataSet["PlayerId"])
          elif k == 1:
              localURL = localURL.replace(j,str(l))
          elif k == 2:
              localURL = localURL.replace(j,m)
        responseIndividualPlayer = requests.get(localURL,headers=headers)

        servingStats = responseIndividualPlayer.json()["Stats"]["ServiceRecordStats"]
        recevingStats = responseIndividualPlayer.json()["Stats"]["ReturnRecordStats"]
        entryDetails = {"Year":l,"courtType":m}
        relevantServingStats = {key: value for key, value in zip(servingStats.keys(), servingStats.values()) if "Percentage" in key }
        relevantReceivingStats = {key: value for key, value in zip(recevingStats.keys(), recevingStats.values()) if "Percentage" in key }
        overallPlayerData = {**filteredDataSet,**entryDetails,**relevantPlayerStats,**relevantServingStats,**relevantReceivingStats}
        #print(json.dumps(overallplayerData, indent=2))
        playerData.append(overallPlayerData)



D643
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all
Grass
Clay
Hard
Carpet
all


In [ ]:
playerDF = pd.DataFrame(playerData)

todayDate = datetime.datetime.now().strftime("%x").replace("/","_")
playerDF.to_csv("playerData_Year_"+year+"_courtType_"+courtType+"_"+todayDate, sep=',', index=False, encoding='utf-8')

In [ ]:
playerDF = pd.DataFrame(playerData)
playerDF.head(120)

,RankNo,Name,Points,Movement,PlayerId,Year,courtType,Age,HeightCm,WeightKg,...,SecondServePointsWonPercentage,BreakPointsSavedPercentage,ServiceGamesWonPercentage,ServicePointsWonPercentage,FirstServeReturnPointsWonPercentage,SecondServeReturnPointsWonPercentage,BreakPointsConvertedPercentage,ReturnGamesWonPercentage,ReturnPointsWonPercentage,TotalPointsWonPercentage
0,1,Novak Djokovic,"9,990",0,D643,2004,Grass,36,188,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Novak Djokovic,"9,990",0,D643,2004,Clay,36,188,77,...,42.0,38.0,61.0,53.0,41.0,47.0,31.0,31.0,43.0,48.0
2,1,Novak Djokovic,"9,990",0,D643,2004,Hard,36,188,77,...,46.0,67.0,69.0,57.0,32.0,42.0,57.0,25.0,35.0,48.0
3,1,Novak Djokovic,"9,990",0,D643,2004,Carpet,36,188,77,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,Novak Djokovic,"9,990",0,D643,2004,all,36,188,77,...,44.0,49.0,63.0,54.0,38.0,46.0,35.0,29.0,41.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,1,Novak Djokovic,"9,990",0,D643,all,Grass,36,188,77,...,56.0,69.0,90.0,70.0,31.0,53.0,40.0,26.0,39.0,54.0
106,1,Novak Djokovic,"9,990",0,D643,all,Clay,36,188,77,...,54.0,63.0,82.0,65.0,37.0,55.0,45.0,35.0,44.0,54.0
107,1,Novak Djokovic,"9,990",0,D643,all,Hard,36,188,77,...,56.0,66.0,87.0,68.0,33.0,56.0,45.0,32.0,42.0,55.0
108,1,Novak Djokovic,"9,990",0,D643,all,Carpet,36,188,77,...,50.0,73.0,83.0,64.0,28.0,54.0,35.0,23.0,37.0,51.0


In [ ]:

#iterate between current year (replace '&season=CURRENTSEASON' with blank) and 2022, collect dates for each year and output player elo ratings to a CSV for each date in each year
import requests
import datetime

seasonDatesURL = 'https://www.ultimatetennisstatistics.com/seasonRankingDates?rankType=ELO_RANK&season=CURRENTSEASON'
rankingInfoURL = 'https://www.ultimatetennisstatistics.com/rankingsTableTable?current=1&rowCount=500&sort%5Brank%5D=asc&searchPhrase=&rankType=ELO_RANK&season=CURRENTSEASON&date=CURRENTDATE&_=1716998656044'
df_playerELO = pd.DataFrame()

#for year in ['2022','2023','2024']:
for year in ['2022','2023']:

    if year == '2024':
        datesURL = seasonDatesURL.replace("&season=CURRENTSEASON","")
    else:
        datesURL = seasonDatesURL.replace("CURRENTSEASON",year)

    datesResponse = requests.get(datesURL)
    try:
        datesJSONresponse = datesResponse.json()
    except requests.exceptions.JSONDecodeError:
        print("jsonError")
        continue
    for date in datesJSONresponse:
        rankingURL = rankingInfoURL.replace('CURRENTSEASON',year)
        formattedDate = datetime.datetime.strptime(date,'%Y-%m-%d').strftime('%d-%m-%Y')

        rankingURL = rankingURL.replace('CURRENTDATE',formattedDate)
        rankingResponse = requests.get(rankingURL)
        try:
            rankingJSONResponse = rankingResponse.json()
        except requests.exceptions.JSONDecodeError:
            print("jsonError")
            continue


        reff = pd.json_normalize(rankingJSONResponse["rows"])
        df = pd.DataFrame(data=reff)
        df["RankingDate"] = formattedDate
        df = df[["rank","name","points","bestPoints","RankingDate"]]

        df_playerELO = pd.concat([df_playerELO, df], ignore_index=True)

df_playerELO.head()